## <u>Table of Contents</u>
* [Introduction](#Introduction)
* [Setup](#Setup)
* [Knowing About the Visualization Configuration File](#Knowing-About-the-Visualization-Configuration-File)
* [Loading and Working with the Sample Data](#Loading-and-Working-with-the-Sample-Data)
* [Dataset Visualization for AV and Agents](#Dataset-Visualization-for-AV-and-Agents)
    * [Visualize the Autonomous Vehicle](#Visualize-the-Autonomous-Vehicle)
    * [Visualizing an Agent](#Visualizing-an-Agent)
    * [Looking at en Entire Scene](#Looking-at-an-Entire-Scene)
* [Some Resource Lists and References](#Some-Resource-Lists-and-References)

## <u>Introduction</u>
* We will try to understand and visualize the data for the [Lyft Motion Prediction for Autonomous Vehicles](https://www.kaggle.com/c/lyft-motion-prediction-autonomous-vehicles/overview) compeition. 
* This competition is quite unique, both in terms of what our objective is and how we handle and interpret the data. So, if you find any mistake or discrepancies anywhere in the notebook, feel free to use the comment section. I will try my best to address them.

### Core Packages
There are two core packages for visualization.
* `rasterization`: This package contains the classes for getting the visual data as multi-channel tensors. After that, this package will help us to turn those multi-channel tensors into RGB images.
    * `rasterize` method: To get the tensor.
    * `to_rgb` method: to convert those tensors into RGB image.
* `visualization`: This package will help us to draw the visual information like trajectories on the converted RGB images.

I hope that everything is understandable upto this point. Still to get to a better idea, let's see how an RGB image after visualization should look like.

**Take a look at the following GIF ([Source](https://self-driving.lyft.com/level5/prediction/))**.
![](https://self-driving.lyft.com/wp-content/uploads/2020/06/motion_dataset_lrg_redux.gif)

So, what can we infer from the above GIF.
* One thing is sure, the red box is the autonomous vehicle.
* The yellow boxes are the agents. We can see vehicular ageints as yellow boxes on the road. But what about other agents like pedestrians? Well, if we look closely, we can see a few yellow dots in the black area. Those maybe pedestrians.
* And the semantic colored lines are the paths along which we have to predict the motions of the exernal agents in this compeition.

Let's try to confirm our theory by looking at the following image.

![](https://self-driving.lyft.com/wp-content/uploads/2020/06/motion_dataset_2-1.png)

Everything seems fine, except that we did not get confirmation on our pedestrian theory. Hopefully, we can skip that part for now.

## <u>Setup</u>
We can either add the L5kit as a utility script or install using pip. Both ways work fine.

In [ ]:
!pip install -U l5kit

Let's do a preliminary setup of all the things that we will need. This includes all the imports and loading the configuration file for visualization.

In [ ]:
%%writefile visualisation_config.yaml
# Config format schema number
format_version: 4

###################
## Model options
model_params:
  model_architecture: "resnet50"

  history_num_frames: 0
  history_step_size: 1
  history_delta_time: 0.1

  future_num_frames: 50
  future_step_size: 1
  future_delta_time: 0.1

###################
## Input raster parameters
raster_params:
  # raster image size [pixels]
  raster_size:
    - 224
    - 224
  # raster's spatial resolution [meters per pixel]: the size in the real world one pixel corresponds to.
  pixel_size:
    - 0.5
    - 0.5
  # From 0 to 1 per axis, [0.5,0.5] would show the ego centered in the image.
  ego_center:
    - 0.25
    - 0.5
  map_type: "py_semantic"

  # the keys are relative to the dataset environment variable
  satellite_map_key: "aerial_map/aerial_map.png"
  semantic_map_key: "semantic_map/semantic_map.pb"
  dataset_meta_key: "meta.json"

  # e.g. 0.0 include every obstacle, 0.5 show those obstacles with >0.5 probability of being
  # one of the classes we care about (cars, bikes, peds, etc.), >=1.0 filter all other agents.
  filter_agents_threshold: 0.5

###################
## Data loader options
val_data_loader:
  key: "scenes/sample.zarr"
  batch_size: 12
  shuffle: False
  num_workers: 16

In [ ]:
import os
import zarr
import numpy as np
import l5kit
import matplotlib.pyplot as plt
import matplotlib

from l5kit.data import PERCEPTION_LABELS
from l5kit.configs import load_config_data
from l5kit.data import ChunkedDataset, LocalDataManager
from l5kit.dataset import EgoDataset, AgentDataset
from l5kit.visualization import draw_trajectory, TARGET_POINTS_COLOR
from l5kit.geometry import transform_points
from l5kit.rasterization import build_rasterizer
from tqdm import tqdm
from prettytable import PrettyTable

In [ ]:
# matplotlib.style.use('ggplot')

In [ ]:
# load the visualizationconfiguration file
cfg = load_config_data('visualisation_config.yaml')

In [ ]:
# set the env variable for the data
os.environ['L5KIT_DATA_FOLDER'] = '../input/lyft-motion-prediction-autonomous-vehicles'

## <u>Knowing About the Visualization Configuration File</u>
* Let get to know what information the configuration file can provide us.

In [ ]:
print(f"Raster Param")
for k, v in cfg['raster_params'].items():
    print(f"{k}: {v}")

So, what do all the above information mean? We will tackle a few important ones.
* `raster_size`: This is size of the image. That is 224x224 pixels.
* `pixel_size`: One pixel corresponds to these many meters. So, along the width and height of the image, one pixel will correspond to 0.5 metres in the real world.
* `ego_enter`: As per the [source](https://github.com/lyft/l5kit/blob/master/examples/visualisation/visualise_data.ipynb) =>  Our raster is centered around an agent, we can move the agent in the image plane with this param. Okay, but still confused what the number mean. If you find an explanation, please tell in the comment section.
* `map_type`: The rasterizer to be used for visualization.

## <u>Loading and Working with the Sample Data</u>

In [ ]:
dm = LocalDataManager()
dataset_path = dm.require(cfg['val_data_loader']['key'])
print(dataset_path)
zarr_dataset = ChunkedDataset(dataset_path)
zarr_dataset.open()
print(zarr_dataset)

The above output gives some useful imformation about the sample scenes.

Now, the following block of code will iterate over all the frames in the sample dataset and plot the trajectory of the autonomous vehicle.

In [ ]:
frames = zarr_dataset.frames
coords = np.zeros((len(frames), 2))
for idx_coord, idx_data in enumerate(tqdm(range(len(frames)), desc='getting centroid to plot trajectory')):
    frame = zarr_dataset.frames[idx_data]
    coords[idx_coord] = frame['ego_translation'][:2]


plt.scatter(coords[:, 0], coords[:, 1], marker='.')
axes = plt.gca()
axes.set_xlim([-2500, 1600])
axes.set_ylim([-2000, 1600])

We can also take a look at the different types of agents and thier labels present in all the frames.

In [ ]:
agents = zarr_dataset.agents
probabilities = agents['label_probabilities']
labels_indexes = np.argmax(probabilities, axis=1)
counts = []
for idx_label, label in enumerate(PERCEPTION_LABELS):
    counts.append(np.sum(labels_indexes == idx_label))
    
table = PrettyTable(field_names=['label', 'counts'])
for count, label in zip(counts, PERCEPTION_LABELS):
    table.add_row([label, count])
print(table)

If I am not very wrong, then the above table shows the differnet labels present in all the frames of the sample data along the `label` column. And the `counts` column shows the total number of times those labeled agents are present in those frames.

## <u>Dataset Visualization for AV and Agents</u>
In this section, we will visulize the autonomous vehicles, thier paths, and the agents on the RGB images.

In [ ]:
rast = build_rasterizer(cfg, dm)
dataset = EgoDataset(cfg, zarr_dataset, rast)

`EgoDataset` will allow us to iterate over the autnomous vehicle annotations.

### Visualize the Autonomous Vehicle

In [ ]:
data = dataset[50]

im = data['image'].transpose(1, 2, 0)
im = dataset.rasterizer.to_rgb(im)
target_positions_pixels = transform_points(data['target_positions'] + data['centroid'][:2], data['world_to_image'])
draw_trajectory(im, target_positions_pixels, data['target_yaws'], TARGET_POINTS_COLOR)

plt.imshow(im[::-1])
plt.show()

The above image is 224x224 pixels as originally intended. Let's make it a bit bigger.

In [ ]:
plt.figure(figsize=(15, 12))
plt.imshow(im[::-1])
plt.show()

That's lot better now. Let's decode the above image.
* The green box: It is the autnomous vehicle.
* The pink line: The trajectory of the automous vehicle.
* The blue box: An agent.
* The yellow lines: The semantic maps.
* But what are the orange outlined boxes: Most prbably traffic posts, but not veru sure.

Okay, now we can take look at the same image but in satellite view.

In [ ]:
cfg['raster_params']['map_type'] = 'py_satellite'
rast = build_rasterizer(cfg, dm)
dataset = EgoDataset(cfg, zarr_dataset, rast)
data = dataset[50]

im = data['image'].transpose(1, 2, 0)
im = dataset.rasterizer.to_rgb(im)
target_positions_pixels = transform_points(data['target_positions'] + data['centroid'][:2], data['world_to_image'])
draw_trajectory(im, target_positions_pixels, data['target_yaws'], TARGET_POINTS_COLOR)

plt.imshow(im[::-1])
plt.show()

Again, increasing the figure size for better visualizations.

In [ ]:
plt.figure(figsize=(15, 12))
plt.imshow(im[::-1])
plt.show()

**Looks like our guess about the orange boxes being traffic posts was right**.

### Visualizing an Agent
**We can do that with the `AgentDataset` class.**

In [ ]:
dataset = AgentDataset(cfg, zarr_dataset, rast)
data = dataset[0]

im = data['image'].transpose(1, 2, 0)
im = dataset.rasterizer.to_rgb(im)
target_positions_pixels = transform_points(data['target_positions'] + data['centroid'][:2], data['world_to_image'])
draw_trajectory(im, target_positions_pixels, data['target_yaws'], TARGET_POINTS_COLOR)

plt.imshow(im[::-1])
plt.show()

In [ ]:
plt.figure(figsize=(15, 12))
plt.imshow(im[::-1])
plt.show()

This time the green box with the trajectory is an agent. **According to the docs it is a pace car and we will see this a lot in this dataset**. I am not very sure what this ***pace car is and what is it purpose***. Do let me know if you have any information.

### Looking at an Entire Scene
Finally we can take a look at the entire seen to get a better idea of everything that is happening around in the dataset.

In [ ]:
from IPython.display import display, clear_output
import PIL

cfg['raster_params']['map_type'] = 'py_semantic'
rast = build_rasterizer(cfg, dm)
dataset = EgoDataset(cfg, zarr_dataset, rast)
scene_idx = 2
indexes = dataset.get_scene_indices(scene_idx)
images = []

for idx in indexes:
    data = dataset[idx]
    im = data['image'].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    target_positions_pixes = transform_points(data['target_positions'] + data['centroid'][:2], data['world_to_image'])
    center_in_pixels = np.asarray(cfg['raster_params']['ego_center']) * cfg['raster_params']['raster_size']
    draw_trajectory(im, target_positions_pixels, data['target_yaws'], TARGET_POINTS_COLOR)
    clear_output(wait=True)
    display(PIL.Image.fromarray(im[::-1]))

**This is it for this notbook. I will be keep updating it if I find something new. Again, this kind of data is very new to me and most probably to many others as well. If you find that there is any wrong information anywhere in this notebook, then please use the comment section to point it out. It will help others as well.**

## <u>Some Resource Lists and References</u>
* [Dataset Formats.](https://github.com/lyft/l5kit/blob/master/data_format.md)
* [GitHub L5kit.](https://github.com/lyft/l5kit).
* [Agent Motion Prediction Config File.](https://github.com/lyft/l5kit/blob/master/examples/agent_motion_prediction/agent_motion_config.yaml)
* [Visualization.](https://github.com/lyft/l5kit/tree/master/examples/visualisation)